<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/RF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de librerias y modelos

In [ ]:
!pip install ebooklib beautifulsoup4 pandas
!pip install stanza

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import re
import stanza
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
stanza.download("es")
nlp = stanza.Pipeline("es", processors="tokenize,pos")

# Definición de funciones

In [ ]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'p\s*\.?\s*e\s*\.?\s*d\s*\.?\s*d\s*\.?\s*o\s*\.?', 'peddo', t, flags=re.IGNORECASE)
    t = re.sub(r's\s*\.?\s*p\s*\.?\s*a\s*\.?\s*d\s*\.?\s*a\.?', 'spada', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def normalize_text_y(t):
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚ0-9¿?,.' -]+", ' ', t)
    t = re.sub(r'p\s*\.?\s*e\s*\.?\s*d\s*\.?\s*d\s*\.?\s*o\s*\.?', 'PEDDO', t, flags=re.IGNORECASE)
    t = re.sub(r's\s*\.?\s*p\s*\.?\s*a\s*\.?\s*d\s*\.?\s*a\.?', 'SPADA', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def convertir_epub_a_pd(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

  f = pd.read_csv('libro_parrafos.csv')
  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  return parrafos

In [ ]:
def categoria_gramatical_stanza(palabra):
    doc = nlp(palabra)
    token = doc.sentences[0].words[0]
    return token.upos

upos2id = {
    "NOUN": 0, #Sustantivo común. Ej: gato, casa, libro, profesor
    "PROPN": 1, #Sustantivo propio. Ej: Argentina, Azul, Google
    "VERB": 2, #Verbo léxico. Ej: comer, hablar, correr
    "ADJ": 3, #Adjetivo. Ej: rápido, azul, brillante
    "ADV": 4, #Adverbio. Ej: rápidamente, muy, cerca
    "PRON": 5, #Pronombre. Ej: yo, tú, él, eso, alguien
    "DET": 6, #Determinante / artículo. Ej: el, la, los, un, ese, mi
    "ADP": 7, #Adposición: preposición o posposición. Ej: de, para, con, sin, sobre
    "SCONJ": 8, #Conjunción subordinante. Ej: que, porque, aunque, si
    "CCONJ": 9, #Conjunción coordinante. Ej: y, o, pero, ni
    "NUM": 10, #Numeral. Ej: uno, dos, 50, tercero
    "INTJ": 11, #Interjección. Ej: ay!, hola!, uf, eh
    "PART": 12, #Partícula gramatical (raro en español). Ejemplos típicos en inglés (not, 's), en español casi no se usa, pero aparece en casos como "sí" enfático.
    "AUX": 13, #Verbo auxiliar. Ej: haber, ser (cuando forman tiempos compuestos: “he comido”, “está hablando”)
    "PUNCT": 14, #Signos de puntuación. Ej: , . ; ! ?
    "SYM": 15, #Símbolos. Ej: $, %, +, =, →
    "X": 16 #Otros / desconocidos / extranjeros. Cualquier cosa que no encaja en ninguna categoría.
}

def indice_categoria_stanza(palabra):
    pos = categoria_gramatical_stanza(palabra)
    return upos2id.get(pos,-1)

In [ ]:
def crearDataSetRFSinEtiquetas(parrafo):
  data_set_RF_sin_etiquetas = pd.DataFrame(columns = ['instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte'])

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []

  for k, parrafo in enumerate(parrafo):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False

    for i, palabra in enumerate(palabras):
      #categoria = indice_categoria_stanza(palabra)
      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria) #cambiar
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        instancia_ids.append(instancia_id)

  data_set_RF_sin_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_sin_etiquetas['token'] = tokens
  data_set_RF_sin_etiquetas['token_id'] = token_ids
  data_set_RF_sin_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_sin_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_sin_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_sin_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_sin_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_sin_etiquetas['es_principio'] = son_principio
  data_set_RF_sin_etiquetas['es_medio'] = son_medio
  data_set_RF_sin_etiquetas['es_final'] = son_final
  data_set_RF_sin_etiquetas['forma_parte'] = forman_parte

  return data_set_RF_sin_etiquetas

In [ ]:
def crearDataSetRFConEtiquetas(parrafos):
  data_set_RF_con_etiquetas = pd.DataFrame(columns = ['palabra_default', 'instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte', 'punt_inicial', 'punt_final', 'capitalización'])

  palabras_default = []

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []
  puntuaciones_iniciales = []
  puntuaciones_finales = []
  capitalizaciones = []

  datos_limpios = parrafos['limpio']
  datos_default = parrafos['default']
  for k, parrafo in enumerate(datos_limpios):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False

    palabras_d = datos_default.iloc[k].split()
    npal = 0
    for i, palabra in enumerate(palabras):
      palabra_default = ' '
      while palabras_d[npal] == '?' or palabras_d[npal] == '¿' or palabras_d[npal] == '.' or palabras_d[npal] == ',':
        if palabras_d[npal] == '¿':
          inicio_pregunta = True
        npal += 1

      if palabras_d[npal] != '?' and palabras_d[npal] != '¿' and palabras_d[npal] != '.' and palabras_d[npal] != ',':
        palabra_default = palabras_d[npal]

      if palabra_default[0] == "¿":
        inicio_pregunta = True

      if palabra_default[-1] == "?":
        puntuacion_final = 3
      elif palabra_default[-1] == ".":
        puntuacion_final = 1
      elif palabra_default[-1] == ",":
        puntuacion_final = 2
      else:
        if npal != len(palabras_d)-1:
          if palabras_d[npal+1] == '?':
            puntuacion_final = 3
          elif palabras_d[npal+1] == '.':
            puntuacion_final = 1
          elif palabras_d[npal+1] == ',':
            puntuacion_final = 2
          else:
            puntuacion_final = 0
        else:
          puntuacion_final = 0

      #categoria = indice_categoria_stanza(palabra)
      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      if palabra_default.islower():
        capitalizacion = 0
      elif palabra_default.istitle():
        capitalizacion = 1
      elif palabra_default.isupper():
        capitalizacion = 3
      else:
        capitalizacion = 2

      npal += 1
      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria) #cambiar
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        if inicio_pregunta:
          puntuaciones_iniciales.append(1)
          inicio_pregunta = False
        else:
          puntuaciones_iniciales.append(0)

        if j == n_tok - 1:
          puntuaciones_finales.append(puntuacion_final)
        else:
          puntuaciones_finales.append(0)

        capitalizaciones.append(capitalizacion)
        instancia_ids.append(instancia_id)
        palabras_default.append(palabra_default)

  data_set_RF_con_etiquetas['palabra_default'] = palabras_default
  data_set_RF_con_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_con_etiquetas['token'] = tokens
  data_set_RF_con_etiquetas['token_id'] = token_ids
  data_set_RF_con_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_con_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_con_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_con_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_con_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_con_etiquetas['es_principio'] = son_principio
  data_set_RF_con_etiquetas['es_medio'] = son_medio
  data_set_RF_con_etiquetas['es_final'] = son_final
  data_set_RF_con_etiquetas['forma_parte'] = forman_parte
  data_set_RF_con_etiquetas['punt_inicial'] = puntuaciones_iniciales
  data_set_RF_con_etiquetas['punt_final'] = puntuaciones_finales
  data_set_RF_con_etiquetas['capitalización'] = capitalizaciones

  return data_set_RF_con_etiquetas

# Carga de datos y creación de data set

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

In [ ]:
parrafos = convertir_epub_a_pd('libro1.epub')

In [ ]:
dataSet_RF_con_etiquetas = crearDataSetRFConEtiquetas(parrafos)
dataSet_RF_sin_etiquetas = crearDataSetRFSinEtiquetas(parrafos['limpio'])

# Atributos para el data set

In [ ]:
dataSet_RF = crearDataSetRF(parrafos['limpio'])
#dataSet_RF = pd.read_csv('dataSetRFSinEtiquetas.csv')

In [ ]:
dataSet_RF.to_csv('dataSetRFSinEtiquetas.csv', index=False)

## Etiqueta 1: capitalización
* 0: todo en minúsculas (ej: “hola”)
* 1: primera letra en mayúscula (ej: “Hola”) (incluye palabras de 1 letra)
* 2: algunas (pero no todas) letras en mayúscula (ej: “McDonald's”  “iPhone”)
* 3: todo en mayúsculas (ej.: “ONU”, “NASA”, “UBA”) (más de una letra)

In [ ]:
f, c = dataSet_RF.shape
f2, c2 = dataSet.shape
#dataSet_RF['capitalizacion'] = dataSet[:f]['capitalización']

In [ ]:
dataSet_RF[dataSet_RF['token_id'] == 10605]

In [ ]:
dataSet_RF[dataSet_RF['token_id'] == 28163]

In [ ]:
dataSet[dataSet['token_id'] ==  30519]

In [ ]:
print(dataSet['token_id'])
print(dataSet_RF['token_id'])

In [ ]:
print(f"Length of dataSet['token_id']: {len(dataSet['token_id'])}")
print(f"Length of dataSet_RF['token_id']: {len(dataSet_RF['token_id'])}")

unique_tokens_dataSet = set(dataSet['token_id'].unique())
unique_tokens_dataSet_RF = set(dataSet_RF['token_id'].unique())

diff_only_in_dataSet = unique_tokens_dataSet - unique_tokens_dataSet_RF
diff_only_in_dataSet_RF = unique_tokens_dataSet_RF - unique_tokens_dataSet

if not diff_only_in_dataSet and not diff_only_in_dataSet_RF:
    print("\nBoth series contain the same unique token_ids, although their lengths might differ.")
else:
    if diff_only_in_dataSet:
        print(f"\nToken_ids present in dataSet but not in dataSet_RF (first 10): {list(diff_only_in_dataSet)[:10]}")
    if diff_only_in_dataSet_RF:
        print(f"\nToken_ids present in dataSet_RF but not in dataSet (first 10): {list(diff_only_in_dataSet_RF)[:10]}")


# Random Forest

In [ ]:
datos_Y['instancia_id'] = datos_X['instancia_id']

datos_X["instancia_id"] = pd.to_numeric(datos_X["instancia_id"], errors="coerce")
datos_Y["instancia_id"] = pd.to_numeric(datos_Y["instancia_id"], errors="coerce")

X_train = datos_X[datos_X['instancia_id'] < 2955]
y_train = datos_Y[datos_Y['instancia_id'] < 2955]

X_test = datos_X[datos_X['instancia_id'] >= 2955]
y_test = datos_Y[datos_Y['instancia_id'] >= 2955]


In [ ]:
X_train['embeddings']

In [ ]:
X_train["embeddings_mean"] = X_train["embeddings"].apply(lambda x: np.mean(x))


In [ ]:
X_test["embeddings_mean"] = X_test["embeddings"].apply(lambda x: np.mean(x))


In [ ]:
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

    #n_estimators cantidad de arboles
    # max_depth altura maxima de cada uno


In [ ]:
y_test

In [ ]:
model.fit(X_train[['embeddings_mean']], y_train.drop(columns=['instancia_id']))


In [ ]:
y_pred = model.predict(X_test[['embeddings_mean']])


In [ ]:
f1_score(y_test['punt_inicial'], y_pred[:,0],average="macro")

In [ ]:
f1_score(y_test['punt_inicial'],y_pred[:,1],average = "macro")

In [ ]:
f1_score(y_test['capitalización'],y_pred[:,2],average = "macro")

### RNN Unidireccional

In [ ]:
class EncoderUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,#como es 2, significa que hay dos bloques de celdas LSTM
            batch_first=True, #(batch, seq, feature)
            dropout=dropout, #dropout probability
            bidirectional=False  # unidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)


In [ ]:
class DecoderUnidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)


        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }

#### Encoder - Decoder

In [ ]:
class ModeloUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloUnidireccional, self).__init__()
        self.encoder = EncoderUnidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderUnidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions

### RNN Bidireccional

In [ ]:
class EncoderBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

In [ ]:
class DecoderBidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


#### Encoder - Decoder bidireccional

In [ ]:
class ModeloBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloBidireccional, self).__init__()
        self.encoder = EncoderBidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderBidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions

### Entrenamiento

In [ ]:
model = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)  # ignorar padding
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for embeddings, labels in dataloader:
        optimizer.zero_grad()

        outputs = model(embeddings)  # diccionario con tus tres salidas

        loss_inicial = criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        loss = loss_inicial + loss_final + loss_cap
        loss.backward()
        optimizer.step()